In [ ]:
import cv2
import numpy as np
import matplotlib.pylab as plt

def create_red_mask() :
    
    # 색상별 영역 지정
    red1 = np.array([0, 50, 50])
    red2 = np.array([15, 255, 255])
    red3 = np.array([165, 50, 50])
    red4 = np.array([180, 255, 255])
    
    # 색상에 따른 마스크 생성
    mask_red1 = cv2.inRange(hsv, red1, red2)
    mask_red2 = cv2.inRange(hsv, red3, red4)
    mask_red = mask_red1 + mask_red2
    
    return mask_red2

# mask_red2 = create_red_mask()

# file_name = 'frame2.png'
def capture_traffic_sign (file_path) :

    img = cv2.imread('./' + file_name)
    img = cv2.resize(img, (img.shape[1]//2, img.shape[0]//2), interpolation = cv2.INTER_AREA)
    img2 = img.copy()
    img3 = img.copy()
    h, w = img.shape[:2]

    # HSV 영상으로 변환
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    img = cv2.bilateralFilter(img,9,105,105)
    r, g, b = cv2.split(img)
    equalize1= cv2.equalizeHist(r)
    equalize2= cv2.equalizeHist(g)
    equalize3= cv2.equalizeHist(b)
    equalize = cv2.merge((equalize1, equalize2, equalize3))
    img = equalize

    numOfLabels, img_label, stats, centroids = cv2.connectedComponentsWithStats(mask_red2)

    for idx, centroid in enumerate(centroids) :
        if stats[idx][0] == 0 and stats[idx][1] == 0 :
            continue

        if np.any(np.isnan(centroid)) :
            continue

        x, y, w, h, area = stats[idx]
        centerX, centerY = int(centroid[0]), int(centroid[1])

        if area > 50 and abs(w-h) < 5 : 
            # 원 검출
            detected_img = img[y:y+h, x:x+w]
            gray_detected_img = cv2.cvtColor(detected_img, cv2.COLOR_BGR2GRAY)
            gray_detected_img = cv2.resize(gray_detected_img, (gray_detected_img.shape[1]*5, gray_detected_img.shape[0]*5))
            circles = cv2.HoughCircles(gray_detected_img, cv2.HOUGH_GRADIENT, 1, 100, param1=50, param2=30, minRadius=10, maxRadius=0)
            if circles is not None :
                circles = np.uint16(np.around(circles))
                for i in circles[0, :] :
                    cv2.circle(gray_detected_img, (i[0], i[1]), i[2], (0, 255, 0), 2)
                    cv2.circle(gray_detected_img, (i[0], i[1]), 2, (0, 0, 255), 5)
                    cv2.rectangle(img2, (x-3, y-3), (x+w+3, y+h+3), (255, 0, 0), 1)                    
                    cv2.imwrite('images/{}_{}.{}'.format(file_name.split('.')[0], idx, file_name.split('.')[1]), img3[y-3:y+h+3, x-3:x+w+3])